## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [0]:
import tensorflow as tf
tf.enable_eager_execution()

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [0]:
#from google.colab import files
#uploaded = files.upload()
#from google.colab import drive
#drive.mount('/gdrive')

In [0]:
import pandas as pd

In [0]:
data_iris = pd.read_csv('Iris.csv')
data_prices = pd.read_csv('prices.csv')

In [0]:

prices = data_prices

### Check all columns in the dataset

In [6]:
prices.columns

Index([u'date', u'symbol', u'open', u'close', u'low', u'high', u'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
prices.drop(columns = ["date","symbol"],axis = 1, inplace =True)

In [8]:
prices.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
prices_1k  = prices.iloc[0:1000,:]


In [10]:
prices_1k["volume"] = prices_1k["volume"]/1000000

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer

#### Convert Training and Test Data to numpy float32 arrays


In [0]:

X = prices_1k.drop(columns = ["volume"])
y = prices_1k["volume"]
transformer = Normalizer()
X = transformer.fit_transform(X)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)

In [0]:
X_train = X_train.astype("float32")
y_train = y_train.astype("float32")
X_test = X_test.astype("float32")
y_test = y_test.astype("float32")

In [0]:
import numpy as np
y_train = np.array([y_train]).transpose()

In [16]:
y_train.shape

(700, 1)

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [17]:
print("Data already normalize")

Data already normalize


## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
#We are initializing weights and Bias with Zero
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

In [19]:
w.dtype

tf.float32

2.Define a function to calculate prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

In [21]:
prediction(X_train,w,b)

<tf.Tensor: id=8, shape=(700, 1), dtype=float32, numpy=
array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [24]:
for i in range(100):
    
    w, b = train(X_train, y_train, w, b)
    print('Current Loss on iteration', i, 
          loss(y_train, prediction(X_train, w, b)).numpy())

('Current Loss on iteration', 0, 223.49039)
('Current Loss on iteration', 1, 221.2885)
('Current Loss on iteration', 2, 219.2592)
('Current Loss on iteration', 3, 217.38904)
('Current Loss on iteration', 4, 215.6655)
('Current Loss on iteration', 5, 214.07701)
('Current Loss on iteration', 6, 212.6131)
('Current Loss on iteration', 7, 211.26398)
('Current Loss on iteration', 8, 210.0206)
('Current Loss on iteration', 9, 208.8747)
('Current Loss on iteration', 10, 207.81863)
('Current Loss on iteration', 11, 206.8453)
('Current Loss on iteration', 12, 205.9484)
('Current Loss on iteration', 13, 205.12177)
('Current Loss on iteration', 14, 204.3599)
('Current Loss on iteration', 15, 203.65776)
('Current Loss on iteration', 16, 203.0107)
('Current Loss on iteration', 17, 202.41435)
('Current Loss on iteration', 18, 201.86476)
('Current Loss on iteration', 19, 201.35823)
('Current Loss on iteration', 20, 200.8914)
('Current Loss on iteration', 21, 200.46118)
('Current Loss on iteration', 2

### Get the shapes and values of W and b

In [34]:
w.shape

TensorShape([Dimension(4), Dimension(1)])

In [35]:
w

<tf.Tensor: id=4398, shape=(4, 1), dtype=float32, numpy=
array([[1.328814 ],
       [1.3804225],
       [1.32355  ],
       [1.3934187]], dtype=float32)>

In [64]:
b.shape

TensorShape([Dimension(1)])

In [65]:
b

<tf.Tensor: id=4401, shape=(1,), dtype=float32, numpy=array([2.7138965], dtype=float32)>

### Model Prediction on 1st Examples in Test Dataset

In [72]:
prediction(X_test[[0]], w, b)

<tf.Tensor: id=10374, shape=(1, 1), dtype=float32, numpy=array([[5.4272947]], dtype=float32)>

In [66]:
X_test[0]

array([0.49957508, 0.49997294, 0.495142  , 0.50525856], dtype=float32)

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [79]:
iris = data_iris
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
#iris['Species_cat'] = iris.Species.astype('category')

In [80]:
#iris['Species_cat'] = iris['Species_cat'].cat.codes
print("Since one hot coding is done not doing label encode")

Since one hot coding is done not doing label encode


In [0]:
iris_new =  pd.get_dummies(iris,prefix=['Species_cat'])

### Splitting the data into feature set and target set

In [0]:
X_iris = iris_new.drop(columns = ["Species_cat_Iris-setosa","Species_cat_Iris-versicolor","Species_cat_Iris-virginica"],axis = 1)

In [0]:
y_iris =  iris_new.iloc[:,[5,6,7]]

In [84]:
y_iris.shape

(150, 3)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [85]:
iris_new.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_cat_Iris-setosa,Species_cat_Iris-versicolor,Species_cat_Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0


In [0]:
X_iris.drop(columns = ["Id"],inplace = True)

In [0]:
transformer = Normalizer()
X_iris = transformer.fit_transform(X_iris)

In [0]:
X_train_iris, X_test_iris, y_train_iris, y_test_iris = train_test_split(X_iris, y_iris, test_size=0.30, random_state=1)

In [0]:
from keras.layers import Dense

### Model Training 

In [0]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Reshape data from 2D to 1D -> 28x28 to 784
#model.add(tf.keras.layers.Reshape((784,),input_shape=(28,28,)))
model.add(tf.keras.layers.Dense(45,input_shape=(4,)))
#model.add(input_dim=(4,))
#Normalize the data
model.add(tf.keras.layers.BatchNormalization())

#Add Dense Layer which provides 10 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3, activation='softmax'))

#Comile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [98]:
model.fit(X_train_iris, y_train_iris, 
          validation_data=(X_test_iris, y_test_iris), 
          epochs=100,
          batch_size=y_train_iris.shape[0])

Train on 105 samples, validate on 45 samples
Epoch 1/100
105/105 [==============================] - 0s 115us/sample - loss: 0.3491 - acc: 0.8762 - val_loss: 0.9326 - val_acc: 0.6889
Epoch 2/100
105/105 [==============================] - 0s 56us/sample - loss: 0.3483 - acc: 0.8762 - val_loss: 0.9310 - val_acc: 0.7111
Epoch 3/100
105/105 [==============================] - 0s 43us/sample - loss: 0.3475 - acc: 0.8762 - val_loss: 0.9293 - val_acc: 0.7111
Epoch 4/100
105/105 [==============================] - 0s 48us/sample - loss: 0.3467 - acc: 0.8762 - val_loss: 0.9277 - val_acc: 0.7111
Epoch 5/100
105/105 [==============================] - 0s 70us/sample - loss: 0.3460 - acc: 0.8762 - val_loss: 0.9261 - val_acc: 0.7111
Epoch 6/100
105/105 [==============================] - 0s 62us/sample - loss: 0.3452 - acc: 0.8762 - val_loss: 0.9245 - val_acc: 0.7111
Epoch 7/100
105/105 [==============================] - 0s 49us/sample - loss: 0.3445 - acc: 0.8762 - val_loss: 0.9229 - val_acc: 0.7111
Ep

### Model Prediction

In [97]:
model.predict_classes(X_test_iris)

array([0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 2, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       1])

### Save the Model

In [106]:
#model.save('Iris_model2.h5')
#!pip install h5py
model.save_weights('Iris_model_weights.h5')
print("Save gave error due some bug in keras version, but file also got saved. In addition using save_wights")

Save gave error due some bug in keras version, but file also got saved. In addition using save_wights


### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?